# Generating a list of Animeplanet usernames
* Looks at followers and following

In [ ]:
import os
import random
import re
import time

from filelock import FileLock

In [ ]:
PROXY_NUMBER = 0
NUM_PROXIES = 1

In [ ]:
name = "get_friend_username"
source = "animeplanet"

In [ ]:
exec(open("web_endpoint_base.py").read())

In [ ]:
LOCK_FILE = f"../../animeplanet_friends.lock"
LOCK = FileLock(LOCK_FILE, timeout=-1)

In [ ]:
def get_users_on_url(url):
    response = call_api(url)
    if response.status_code in [404]:
        logging.warning(f"Error {response} received when handling {url}")
        return set()
    if not response.ok:
        logging.warning(f"Error {response} received when handling {url}")
        return set()
    urls = re.findall("""/users/[^"/#%?.'><]+""", response.text)
    users = {x[len("/users/") :] for x in urls}
    return users

In [ ]:
def get_users_on_base_url(base_url):
    users = set()
    page = 1
    while True:
        url = base_url + f"?page={page}"
        new_users = get_users_on_url(url)
        if len(users) == len(users | new_users):
            break
        users |= new_users
        page += 1
    return users

In [ ]:
def get_users(user):
    users = [
        get_users_on_base_url(f"https://www.anime-planet.com/users/{user}/{x}")
        for x in ["followers", "following"]
    ]
    return set().union(*users)

In [ ]:
def reset_state():
    usernames = set()
    if os.path.exists("friend_usernames.txt"):
        with open("friend_usernames.txt") as f:
            usernames = {x.strip() for x in f.readlines() if x.strip()}
    recent = set()
    if os.path.exists("recent_usernames.txt"):
        with open("recent_usernames.txt") as f:
            recent = {x.strip() for x in f.readlines() if x.strip()}
    visited = set()
    if os.path.exists("friend_visited.txt"):
        with open("friend_visited.txt") as f:
            visited = {x.strip() for x in f.readlines() if x.strip()}
    logging.info(
        f"Starting with {len(usernames | recent)} stored usernames and {len(visited)} visited usernames"
    )
    new_users = list((usernames | recent) - visited)
    old_users = list(visited)
    random.shuffle(new_users)
    random.shuffle(old_users)
    order = (new_users + old_users)[:1000]
    if not order:
        timeout = 600
        logging.info(f"No users found. Retrying in {timeout} seconds")
        time.sleep(timeout)
        return reset_state()
    return usernames, recent, visited, order

In [ ]:
while True:
    usernames, recent, visited, order = reset_state()
    for user in order:
        users = get_users(user)
        visited.add(user)
        usernames |= users
        if should_save("users", max_iters=500) or user == order[-1]:
            with LOCK:
                atomic_to_csv(sorted(list(usernames)), "friend_usernames.txt")
                atomic_to_csv(sorted(list(visited)), "friend_visited.txt")
                logging.info(
                    f"Successfully written {len(usernames)} users and visited {len(visited)} usernames"
                )
    time.sleep(1)